In [ ]:
# Define the base path to your dataset on Google Drive
drive_base_path = '/content/drive/MyDrive/DDR_dataset'

In [ ]:
#Mount Google Drive (already mounted)
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Create output directory
output_dir = "/content/DDR_dataset"
os.makedirs(output_dir, exist_ok=True)

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import zipfile
import torch
from google.colab import drive

In [5]:
# Use 7zip to extract the split archives
print("Extracting split zip files using 7zip...")
# Combine first part with 7z (handles split archives correctly)
!7z x '/content/drive/MyDrive/DDR_dataset/Copy of DDR-dataset.zip.001' -o'/content/DDR_dataset'

Extracting split zip files using 7zip...

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/DDR_dataset/                                             1 file, 1572864000 bytes (1500 MiB)

Extracting archive: /content/drive/MyDrive/DDR_dataset/Copy of DDR-dataset.zip.001
  0% 1 Open            22% 4096 Open              --
Path = /content/drive/MyDrive/DDR_dataset/Copy of DDR-dataset.zip.001
Type = Split
Physical Size = 1572864000
Volumes = 10
Total Physical Size = 15142640333
----
Path = Copy of DDR-dataset.zip
Size = 15142640333
--
Path = Copy of DDR-dataset.zip
Type = zip
Physical Size = 15142640333
64-bit = +

  0%      0% 32 - D

In [6]:
!pip install segmentation-models-pytorch albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [ ]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import gc

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Enhanced image preprocessing functions
def enhance_contrast(image):
    """Apply advanced contrast enhancement optimized for MA detection"""
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Apply stronger CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    # Merge channels and convert back to RGB
    lab_clahe = cv2.merge((l_clahe, a, b))
    image_enhanced = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2RGB)

    # Additional processing for green channel which shows MAs better
    green_channel = image_enhanced[:,:,1].astype(np.uint8)

    # Apply contrast stretching to green channel
    minval = np.percentile(green_channel, 2)
    maxval = np.percentile(green_channel, 98)

    # Stretch contrast
    if maxval > minval:
        stretched = np.clip((green_channel - minval) * 255.0 / (maxval - minval), 0, 255).astype(np.uint8)
        image_enhanced[:,:,1] = stretched

    return image_enhanced

def remove_vessels(image):
    """Apply improved vessel removal with advanced techniques"""
    # Extract green channel (best for vessel visualization)
    green_channel = image[:, :, 1].copy()

    # Create multi-scale kernel approach for better vessel detection
    kernels = [
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)),
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)),
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    ]

    # Create combined vessel mask using multiple scales
    vessel_mask = np.zeros_like(green_channel)

    for kernel in kernels:
        # Apply top-hat transform to detect vessels
        tophat = cv2.morphologyEx(green_channel, cv2.MORPH_TOPHAT, kernel)

        # Threshold to create vessel mask with more aggressive threshold
        _, curr_mask = cv2.threshold(tophat, 20, 255, cv2.THRESH_BINARY)
        curr_mask = cv2.dilate(curr_mask, kernel, iterations=1)

        # Combine with main mask
        vessel_mask = cv2.bitwise_or(vessel_mask, curr_mask)

    # Create 3D mask from vessel mask
    vessel_mask_3d = np.stack([vessel_mask, vessel_mask, vessel_mask], axis=2)

    # Remove vessels - reduce intensity in vessel regions more aggressively
    image_no_vessels = image.copy()
    image_no_vessels[vessel_mask_3d > 0] = image_no_vessels[vessel_mask_3d > 0] * 0.5  # Reduced from 0.6 to 0.5

    return image_no_vessels

def enhance_ma_visibility(image):
    """Special processing to enhance microaneurysms visibility"""
    # Extract green channel (best for MA visualization)
    green_channel = image[:, :, 1].copy().astype(np.float32)

    # Apply Gaussian blur to reduce noise
    smoothed = cv2.GaussianBlur(green_channel, (3, 3), 0)

    # Apply unsharp masking to enhance small details like MAs
    sharp = green_channel - 0.7 * smoothed

    # Normalize back to 0-255 range
    sharp = cv2.normalize(sharp, None, 0, 255, cv2.NORM_MINMAX)

    # Replace green channel with enhanced version
    enhanced = image.copy()
    enhanced[:, :, 1] = sharp.astype(np.uint8)

    return enhanced

def apply_multi_scale_enhancement(image):
    """Apply multi-scale enhancement techniques to better highlight different lesion types"""
    # Apply basic contrast enhancement
    enhanced = enhance_contrast(image)

    # Apply vessel removal
    no_vessels = remove_vessels(enhanced)

    # Apply MA-specific enhancement
    ma_enhanced = enhance_ma_visibility(no_vessels)

    return ma_enhanced

# Custom Dataset class with improved preprocessing pipeline
class AdvancedLesionSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dirs, transform=None, apply_preprocessing=True):
        self.image_dir = image_dir
        self.mask_dirs = mask_dirs  # dict with lesion names and folder paths
        self.image_files = sorted(os.listdir(image_dir))
        self.transform = transform
        self.apply_preprocessing = apply_preprocessing

        # Pre-calculate class distribution for weighted sampling
        self.lesion_weights = self._calculate_lesion_weights()

    def _calculate_lesion_weights(self):
        """Calculate weights based on lesion presence in each image"""
        weights = np.ones(len(self.image_files))

        # Count images with microaneurysms for weighted sampling
        for idx, img_file in enumerate(self.image_files):
            ma_mask_path = os.path.join(self.mask_dirs['MA'], img_file.replace('.jpg', '.tif'))
            if os.path.exists(ma_mask_path):
                mask = cv2.imread(ma_mask_path, cv2.IMREAD_GRAYSCALE)
                if mask is not None and np.any(mask > 0):
                    # Assign higher weight to images with MA
                    weights[idx] = 5.0  # Increased from 3.0 to 5.0

        return weights

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image: {image_path}")
            image = np.zeros((512, 512, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply advanced preprocessing
        if self.apply_preprocessing:
            # Apply multi-scale enhancement
            image = apply_multi_scale_enhancement(image)

        # Load mask for each lesion type
        mask_channels = []
        for key in ['MA', 'EX', 'SE', 'HE']:
            mask_path = os.path.join(self.mask_dirs[key], self.image_files[idx].replace('.jpg', '.tif'))

            if os.path.exists(mask_path):
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                if mask is None:
                    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
                else:
                    if mask.shape[0] != image.shape[0] or mask.shape[1] != image.shape[1]:
                        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

                    # For MA, dilate the mask more aggressively
                    if key == 'MA':
                        kernel = np.ones((5, 5), np.uint8)  # Increased from 3x3 to 5x5
                        mask = cv2.dilate(mask, kernel, iterations=2)  # Increased from 1 to 2 iterations

                    mask = (mask > 0).astype(np.uint8)
            else:
                mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

            mask_channels.append(mask)

        # Create multi-class segmentation mask with priority for small lesions
        multi_class_mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        # Apply masks with MA having highest priority
        # Order: EX, SE, HE, MA (increasing priority)
        for class_idx, lesion_idx in [(2, 1), (3, 2), (4, 3), (1, 0)]:  # (class ID, array index)
            multi_class_mask[mask_channels[lesion_idx] > 0] = class_idx

        # Apply transformations
        if self.transform:
            augmented = self.transform(image=image, mask=multi_class_mask)
            image = augmented['image']
            multi_class_mask = augmented['mask']

        return image, multi_class_mask

# Enhanced augmentation pipeline specifically optimized for MA detection
def get_advanced_transforms(phase, img_size=512):
    if phase == 'train':
        return A.Compose([
            A.Resize(height=img_size, width=img_size),
            # Spatial augmentations with increased probabilities
            A.HorizontalFlip(p=0.6),  # Increased from 0.5 to 0.6
            A.VerticalFlip(p=0.6),    # Increased from 0.5 to 0.6
            A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=20, border_mode=cv2.BORDER_CONSTANT, p=0.7),

            # Careful with crop to preserve small lesions
            A.RandomCrop(height=int(img_size*0.9), width=int(img_size*0.9), p=0.4),
            A.Resize(height=img_size, width=img_size, p=1.0),

            # More aggressive color augmentations
            A.OneOf([
                A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.4, p=1.0),  # Increased contrast limit
                A.CLAHE(clip_limit=5.0, tile_grid_size=(8, 8), p=1.0),  # Increased clip limit
            ], p=0.8),  # Increased probability from 0.7 to 0.8

            # Normalize
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:  # validation or test
        return A.Compose([
            A.Resize(height=img_size, width=img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

# Enhanced Combined Loss with modified weighting
class CombinedLoss(nn.Module):
    def __init__(self, class_weights=None):
        super(CombinedLoss, self).__init__()
        self.ce_loss = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, inputs, targets):
        # Cross entropy loss
        ce_loss = self.ce_loss(inputs, targets)

        # Dice loss component
        inputs_soft = torch.softmax(inputs, dim=1)

        # One-hot encode targets
        targets_one_hot = torch.zeros_like(inputs_soft)
        targets_one_hot = targets_one_hot.scatter_(1, targets.unsqueeze(1), 1)

        # Calculate dice loss for each class
        dice_per_class = []
        for i in range(inputs.shape[1]):
            dice_numerator = 2 * (inputs_soft[:, i] * targets_one_hot[:, i]).sum() + 1e-6
            dice_denominator = inputs_soft[:, i].sum() + targets_one_hot[:, i].sum() + 1e-6
            dice_per_class.append(1 - (dice_numerator / dice_denominator))

        # Weight class 1 (MA) more heavily in the dice loss
        dice_weights = torch.tensor([1.0, 4.0, 1.5, 1.5, 1.5], device=inputs.device)  # Increased MA weight from 3.0 to 4.0
        dice_loss = sum(d * w for d, w in zip(dice_per_class, dice_weights)) / sum(dice_weights)

        # Combined loss with modified ratio
        return 0.4 * ce_loss + 0.6 * dice_loss  # Changed from 0.5/0.5 to 0.4/0.6

# Attention modules specialized for MA detection
class MAAttention(nn.Module):
    def __init__(self, in_channels):
        super(MAAttention, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=False)
        self.conv2 = nn.Conv2d(32, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        return self.sigmoid(x)

# Enhanced UNet model with two-stage MA attention
class EnhancedUNet(nn.Module):
    def __init__(self, encoder_name='efficientnet-b4', encoder_weights='imagenet', in_channels=3, classes=5):
        super(EnhancedUNet, self).__init__()

        # Base UNet model
        self.model = smp.Unet(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=classes,
            decoder_attention_type='scse'  # Use spatial and channel squeeze & excitation
        )

        # Two-stage MA attention
        self.ma_attention1 = MAAttention(classes)
        self.ma_attention2 = MAAttention(classes)

    def forward(self, x):
        # Get base output
        features = self.model(x)

        # Two-stage MA attention
        ma_attention1 = self.ma_attention1(features)
        ma_channel = features[:, 1:2].clone()
        ma_enhanced = ma_channel * ma_attention1

        # Second stage attention
        ma_attention2 = self.ma_attention2(features)
        ma_enhanced = ma_enhanced * ma_attention2

        # Replace MA channel
        output = features.clone()
        output[:, 1:2] = ma_enhanced

        return output

# Post-processing function for better predictions
def post_process_prediction(pred_mask, class_idx):
    """Apply post-processing to improve prediction quality"""
    if class_idx == 1:  # MA
        # For MA, remove very small components
        num_labels, labels = cv2.connectedComponents(pred_mask.astype(np.uint8))
        for label in range(1, num_labels):
            component = (labels == label).astype(np.uint8)
            if np.sum(component) < 3:  # Remove tiny components
                pred_mask[component > 0] = 0

        # Apply closing to fill small gaps
        kernel = np.ones((3, 3), np.uint8)
        pred_mask = cv2.morphologyEx(pred_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    elif class_idx in [2, 3, 4]:  # EX, SE, HE
        # For larger lesions, smooth boundaries
        kernel = np.ones((5, 5), np.uint8)
        pred_mask = cv2.morphologyEx(pred_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
        pred_mask = cv2.morphologyEx(pred_mask, cv2.MORPH_OPEN, kernel, iterations=1)

    return pred_mask

# IoU computation function with post-processing
def compute_iou(outputs, masks, apply_post_processing=True):
    """
    Compute IoU for each class with post-processing option
    outputs: model predictions (B, C, H, W)
    masks: ground truth (B, H, W)
    """
    # Get predicted class
    if outputs.shape[1] > 1:  # Multi-class case
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
    else:  # Binary case
        preds = (outputs > 0.5).cpu().numpy()

    true_masks = masks.cpu().numpy()

    ious = []
    # Calculate IoU for each lesion class (skip background)
    for c in range(1, 5):  # 1=MA, 2=EX, 3=SE, 4=HE
        all_batch_ious = []

        for b in range(preds.shape[0]):  # Process each batch item
            pred_mask = (preds[b] == c).astype(np.float32)
            true_mask = (true_masks[b] == c).astype(np.float32)

            # Apply post-processing if requested
            if apply_post_processing:
                pred_mask = post_process_prediction(pred_mask, c)

            # Calculate intersection and union
            intersection = np.sum(pred_mask * true_mask)
            union = np.sum(pred_mask) + np.sum(true_mask) - intersection + 1e-7

            # Check if there are any true masks for this class
            if np.sum(true_mask) == 0:
                # If there's no true mask but prediction is also empty, this is correct
                if np.sum(pred_mask) == 0:
                    all_batch_ious.append(1.0)
                else:
                    all_batch_ious.append(0.0)
            else:
                all_batch_ious.append(intersection / union)

        # Average IoU for this class across all batch items
        ious.append(np.mean(all_batch_ious))

    return ious

# Function to compute mAP with post-processing
def compute_map(outputs, masks, thresholds=None, apply_post_processing=True):
    """
    Compute mean Average Precision (mAP) for each class with post-processing option
    outputs: model predictions after softmax (B, C, H, W)
    masks: ground truth (B, H, W)
    thresholds: list of threshold values to consider for AP calculation
    """
    if thresholds is None:
        thresholds = np.arange(0.5, 1.0, 0.05)

    # Apply softmax to get probabilities
    if isinstance(outputs, torch.Tensor):
        probs = torch.softmax(outputs, dim=1)
        probs = probs.cpu().numpy()
    else:
        probs = outputs

    # Convert masks to numpy if they're tensors
    if isinstance(masks, torch.Tensor):
        masks_np = masks.cpu().numpy()
    else:
        masks_np = masks

    # Calculate AP for each class at each threshold
    aps = []
    # Skip background class (index 0)
    for c in range(1, probs.shape[1]):
        ap_class = []
        for threshold in thresholds:
            # Calculate batch-wise AP and average
            batch_f1s = []

            for b in range(probs.shape[0]):  # Process each batch item
                # Binary prediction for this class at this threshold
                pred_mask = (probs[b, c] > threshold).astype(np.uint8)
                true_mask = (masks_np[b] == c).astype(np.uint8)

                # Apply post-processing if requested
                if apply_post_processing:
                    pred_mask = post_process_prediction(pred_mask, c)

                # Check if there are any true masks for this class
                if np.sum(true_mask) == 0:
                    if np.sum(pred_mask) == 0:
                        # True negative - no masks, no predictions
                        batch_f1s.append(1.0)
                    else:
                        # False positive - no masks but predicted some
                        batch_f1s.append(0.0)
                    continue

                # Compute TP, FP, FN
                tp = np.sum(pred_mask * true_mask)
                fp = np.sum(pred_mask) - tp
                fn = np.sum(true_mask) - tp

                # Compute precision and recall
                precision = tp / (tp + fp + 1e-7)
                recall = tp / (tp + fn + 1e-7)

                # Compute F1 score
                f1 = 2 * precision * recall / (precision + recall + 1e-7)
                batch_f1s.append(f1)

            # Average F1 across all batch items
            ap_class.append(np.mean(batch_f1s))

        # Average AP over all thresholds for this class
        if len(ap_class) > 0:
            aps.append(np.mean(ap_class))
        else:
            aps.append(0.0)

    # Return AP for each class and mAP
    return aps, np.mean(aps)

# Training function with extended patience
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=200):
    # Create directories for output
    os.makedirs('models', exist_ok=True)
    os.makedirs('visuals', exist_ok=True)

    # Initialize tracking variables
    best_miou = 0.0
    best_map = 0.0
    history = {'train_loss': [], 'val_loss': [], 'val_iou': [], 'val_map': []}
    patience = 40  # Increased from 20 to 35
    patience_counter = 0

    # Training loop
    for epoch in range(1, num_epochs + 1):
        # Training phase
        model.train()
        running_loss = 0.0

        for i, (images, masks) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} (Training)")):
            # Clear memory
            torch.cuda.empty_cache()

            images = images.to(device)
            masks = masks.to(device).long()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            # Step the scheduler if it's OneCycleLR
            if isinstance(scheduler, torch.optim.lr_scheduler.OneCycleLR):
                scheduler.step()

            running_loss += loss.item()

            # Delete intermediate tensors to free memory
            del images, masks, outputs, loss
            torch.cuda.empty_cache()

        train_loss = running_loss / len(train_loader)
        history['train_loss'].append(train_loss)

        # Validation phase
        model.eval()
        val_loss = 0.0
        all_ious = []
        all_maps = []

        with torch.no_grad():
            for images, masks in tqdm(val_loader, desc=f"Epoch {epoch}/{num_epochs} (Validation)"):
                # Clear memory
                torch.cuda.empty_cache()

                images = images.to(device)
                masks = masks.to(device).long()

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()

                # Calculate IoU with post-processing
                ious = compute_iou(outputs, masks, apply_post_processing=True)
                all_ious.append(ious)

                # Calculate mAP with post-processing
                maps, _ = compute_map(outputs, masks, apply_post_processing=True)
                all_maps.append(maps)

                # Delete intermediate tensors to free memory
                del images, masks, outputs, loss
                torch.cuda.empty_cache()

        val_loss /= len(val_loader)
        history['val_loss'].append(val_loss)

        # Calculate mean IoU for each class
        mean_ious = np.mean(all_ious, axis=0)
        miou = np.mean(mean_ious)
        history['val_iou'].append(mean_ious.tolist())

        # Calculate mean AP for each class
        mean_maps = np.mean(all_maps, axis=0)
        mmap = np.mean(mean_maps)
        history['val_map'].append(mean_maps.tolist())

        # Update learning rate if not using OneCycleLR
        if not isinstance(scheduler, torch.optim.lr_scheduler.OneCycleLR):
            scheduler.step(val_loss)

        # Print results
        print(f"Epoch {epoch}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        print(f"IoU for MA: {mean_ious[0]:.4f}")
        print(f"IoU for EX: {mean_ious[1]:.4f}")
        print(f"IoU for SE: {mean_ious[2]:.4f}")
        print(f"IoU for HE: {mean_ious[3]:.4f}")
        print(f"Mean IoU: {miou:.4f}")

        print(f"AP for MA: {mean_maps[0]:.4f}")
        print(f"AP for EX: {mean_maps[1]:.4f}")
        print(f"AP for SE: {mean_maps[2]:.4f}")
        print(f"AP for HE: {mean_maps[3]:.4f}")
        print(f"Mean AP: {mmap:.4f}")

        # Save best model based on mIoU
        if miou > best_miou:
            best_miou = miou
            patience_counter = 0

            # Save model state dict
            torch.save(model.state_dict(), 'models/best_model_miou.pth')
            print(f"Model saved with Mean IoU: {best_miou:.4f}")

        # Also save best model based on mAP
        if mmap > best_map:
            best_map = mmap

            # Save model state dict
            torch.save(model.state_dict(), 'models/best_model_map.pth')
            print(f"Model saved with Mean AP: {best_map:.4f}")
        else:
            patience_counter += 1

        # Early stopping
        if patience_counter >= patience:
            print(f"Early stopping triggered. No improvement for {patience} epochs.")
            break

    print(f"Best Mean IoU achieved: {best_miou:.4f}")
    print(f"Best Mean AP achieved: {best_map:.4f}")
    return history, best_miou, best_map

def main():
    # Paths
    train_img_dir = "/content/DDR_dataset/DDR-dataset/lesion_segmentation/train/image"
    val_img_dir = "/content/DDR_dataset/DDR-dataset/lesion_segmentation/valid/image"

    train_mask_dirs = {
        "MA": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/train/label/MA",
        "EX": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/train/label/EX",
        "SE": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/train/label/SE",
        "HE": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/train/label/HE"
    }

    val_mask_dirs = {
        "MA": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/valid/segmentation label/MA",
        "EX": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/valid/segmentation label/EX",
        "SE": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/valid/segmentation label/SE",
        "HE": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/valid/segmentation label/HE"
    }

    # Create datasets
    print("Setting up training dataset from", train_img_dir)
    train_dataset = AdvancedLesionSegmentationDataset(
        image_dir=train_img_dir,
        mask_dirs=train_mask_dirs,
        transform=get_advanced_transforms('train'),
        apply_preprocessing=True
    )

    print("Setting up validation dataset from", val_img_dir)
    val_dataset = AdvancedLesionSegmentationDataset(
        image_dir=val_img_dir,
        mask_dirs=val_mask_dirs,
        transform=get_advanced_transforms('val'),
        apply_preprocessing=True
    )

    # Create weighted sampler for training to focus more on images with MA lesions
    sampler = WeightedRandomSampler(
        weights=train_dataset.lesion_weights,
        num_samples=len(train_dataset),
        replacement=True
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=20,
        sampler=sampler,
        num_workers=14,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=20,
        shuffle=False,
        num_workers=14,
        pin_memory=True
    )

    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")

    # Check first batch
    for images, masks in train_loader:
        print(f"Checking first batch:")
        print(f"Images shape: {images.shape}")
        print(f"Masks shape: {masks.shape}")
        print(f"Unique values in masks: {torch.unique(masks).tolist()}")
        print(f"Mask distribution:")
        for c in range(5):
            count = (masks == c).sum().item()
            pct = count / masks.numel() * 100
            print(f"  Class {c}: {count} pixels ({pct:.4f}%)")
        break

    # Try a better encoder
    encoder_name = "resnext101_32x8d"  # Alternative stronger encoder instead of efficientnet-b4
    print(f"Training model with encoder: {encoder_name}")

    # Clean CUDA memory
    torch.cuda.empty_cache()
    gc.collect()

    # Create model
    model = EnhancedUNet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=5
    ).to(device)

    # Strongly weighted class weights for extreme imbalance - focus on MA
    class_weights = torch.FloatTensor([1.0, 150.0, 25.0, 25.0, 25.0]).to(device)  # Increased MA weight from 120 to 150
    criterion = CombinedLoss(class_weights=class_weights)

    # Create optimizer with learning rate
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

    # Use OneCycleLR with modified parameters
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=8e-4,  # Increased from 5e-4 to 8e-4
        epochs=200,   # Increased epochs
        steps_per_epoch=len(train_loader),
        pct_start=0.15,  # Increased from 0.1 to 0.15
        div_factor=10.0,
        final_div_factor=100.0
    )

    # Train the model
    print("-" * 40)
    print("Starting training...")
    print("-" * 40)

    history, best_miou, best_map = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=200  # Increased from 100 to 200
    )

    # Final evaluation with best model
    # Load best model based on MAP (usually better for practical applications)
    best_model = EnhancedUNet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=5
    ).to(device)

    best_model.load_state_dict(torch.load('models/best_model_map.pth'))
    best_model.eval()

    # Standard evaluation without TTA
    all_ious = []
    all_maps = []

    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc="Final Evaluation"):
            # Clear memory
            torch.cuda.empty_cache()

            images = images.to(device)
            masks = masks.to(device).long()

            # Forward pass
            outputs = best_model(images)

            # Calculate IoU with post-processing
            ious = compute_iou(outputs, masks, apply_post_processing=True)
            all_ious.append(ious)

            # Calculate mAP with post-processing
            maps, _ = compute_map(outputs, masks, apply_post_processing=True)
            all_maps.append(maps)

            # Delete intermediate tensors to free memory
            del images, masks, outputs
            torch.cuda.empty_cache()

    # Calculate final metrics
    final_ious = np.mean(all_ious, axis=0)
    final_miou = np.mean(final_ious)

    final_maps = np.mean(all_maps, axis=0)
    final_mmap = np.mean(final_maps)

    print("\nFinal Evaluation Results:")
    print(f"IoU for MA: {final_ious[0]:.4f}")
    print(f"IoU for EX: {final_ious[1]:.4f}")
    print(f"IoU for SE: {final_ious[2]:.4f}")
    print(f"IoU for HE: {final_ious[3]:.4f}")
    print(f"Mean IoU: {final_miou:.4f}")

    print(f"AP for MA: {final_maps[0]:.4f}")
    print(f"AP for EX: {final_maps[1]:.4f}")
    print(f"AP for SE: {final_maps[2]:.4f}")
    print(f"AP for HE: {final_maps[3]:.4f}")
    print(f"Mean AP: {final_mmap:.4f}")

    # Plot training history
    plt.figure(figsize=(15, 10))

    # Plot loss
    plt.subplot(2, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    # Plot IoU
    plt.subplot(2, 2, 2)
    for i, lesion in enumerate(['MA', 'EX', 'SE', 'HE']):
        iou_values = [epoch_ious[i] for epoch_ious in history['val_iou']]
        plt.plot(iou_values, label=f'{lesion} IoU')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.title('Validation IoU per Class')

    # Plot mAP
    plt.subplot(2, 2, 3)
    for i, lesion in enumerate(['MA', 'EX', 'SE', 'HE']):
        map_values = [epoch_maps[i] for epoch_maps in history['val_map']]
        plt.plot(map_values, label=f'{lesion} AP')
    plt.xlabel('Epoch')
    plt.ylabel('AP')
    plt.legend()
    plt.title('Validation AP per Class')

    # Plot average metrics
    plt.subplot(2, 2, 4)
    miou_values = [np.mean(epoch_ious) for epoch_ious in history['val_iou']]
    mmap_values = [np.mean(epoch_maps) for epoch_maps in history['val_map']]
    plt.plot(miou_values, label='Mean IoU')
    plt.plot(mmap_values, label='Mean AP')
    plt.xlabel('Epoch')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.title('Mean IoU vs Mean AP')

    plt.tight_layout()
    plt.savefig('training_history.png')

    # Generate some visual examples for final output
    print("\nGenerating visual examples for qualitative assessment...")

    # Create a directory for visualizations if it doesn't exist
    os.makedirs('visualizations', exist_ok=True)

    # Define a colormap for visualization with alpha transparency
    def create_color_map_with_alpha():
        # Color map with RGBA (the last number is alpha/transparency)
        color_map = np.zeros((5, 4), dtype=np.uint8)
        color_map[0] = [0, 0, 0, 0]        # Background (transparent)
        color_map[1] = [255, 0, 0, 180]    # MA (red with transparency)
        color_map[2] = [0, 255, 0, 180]    # EX (green with transparency)
        color_map[3] = [255, 255, 0, 180]  # SE (yellow with transparency)
        color_map[4] = [0, 0, 255, 180]    # HE (blue with transparency)
        return color_map

    color_map = create_color_map_with_alpha()

    # Function to create colored segmentation overlay
    def create_segmentation_overlay(image, mask):
        # Create an empty RGBA overlay
        overlay = np.zeros((mask.shape[0], mask.shape[1], 4), dtype=np.uint8)

        # Fill in the overlay with colored regions according to mask
        for i in range(1, 5):  # Skip background (index 0)
            overlay[mask == i] = color_map[i]

        # Convert RGB image to RGBA by adding an alpha channel
        image_rgba = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)
        image_rgba[:, :, 0:3] = image
        image_rgba[:, :, 3] = 255  # Full opacity for base image

        # Alpha blending
        alpha = overlay[:, :, 3:4] / 255.0
        overlay_rgb = overlay[:, :, 0:3]
        image_rgb = image_rgba[:, :, 0:3]

        # Blend images
        blended = image_rgb * (1 - alpha) + overlay_rgb * alpha

        return blended.astype(np.uint8)

    # Visualize some examples with model predictions
    with torch.no_grad():
        for idx, (images, masks) in enumerate(val_loader):
            if idx >= 5:  # Only visualize 5 examples
                break

            images = images.to(device)
            masks = masks.to(device).long()

            # Get predictions (without TTA)
            outputs = best_model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            true_masks = masks.cpu().numpy()

            # Create colorized visualizations
            for b in range(min(2, preds.shape[0])):  # Visualize first 2 images in the batch
                # Get original image
                img = images[b].cpu().numpy()
                img = np.transpose(img, (1, 2, 0))  # CHW to HWC
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                img = np.clip(img, 0, 1) * 255
                img = img.astype(np.uint8)

                # Create overlaid visualizations
                pred_overlay = create_segmentation_overlay(img, preds[b])
                true_overlay = create_segmentation_overlay(img, true_masks[b])

                # Create visualization with original, ground truth overlay, and prediction overlay
                vis_img = np.hstack([img, true_overlay, pred_overlay])

                # Save visualization
                cv2.imwrite(f'visualizations/example_{idx}_{b}.png', cv2.cvtColor(vis_img, cv2.COLOR_RGB2BGR))

    print("\nTraining and evaluation completed! Visual examples saved in 'visualizations' directory.")
    return best_model, history, final_miou, final_mmap

if __name__ == "__main__":
    main()

Using device: cuda
Setting up training dataset from /content/DDR_dataset/DDR-dataset/lesion_segmentation/train/image


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Setting up validation dataset from /content/DDR_dataset/DDR-dataset/lesion_segmentation/valid/image
Training dataset size: 383
Validation dataset size: 149


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 14 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Checking first batch:
Images shape: torch.Size([20, 3, 512, 512])
Masks shape: torch.Size([20, 512, 512])
Unique values in masks: [0, 1, 2, 3, 4]
Mask distribution:
  Class 0: 5221619 pixels (99.5945%)
  Class 1: 4972 pixels (0.0948%)
  Class 2: 8097 pixels (0.1544%)
  Class 3: 979 pixels (0.0187%)
  Class 4: 7213 pixels (0.1376%)
Training model with encoder: resnext101_32x8d


...

Epoch 180/200 (Validation): 100%|██████████| 8/8 [00:45<00:00,  5.63s/it]


Epoch 180/200
Train Loss: 0.4236, Val Loss: 0.5881
IoU for MA: 0.0543
IoU for EX: 0.5342
IoU for SE: 0.5532
IoU for HE: 0.4072
Mean IoU: 0.3872
AP for MA: 0.1187
AP for EX: 0.5521
AP for SE: 0.5930
AP for HE: 0.4663
Mean AP: 0.4325


Epoch 181/200 (Validation): 100%|██████████| 8/8 [00:42<00:00,  5.36s/it]


Epoch 181/200
Train Loss: 0.4168, Val Loss: 0.5932
IoU for MA: 0.0598
IoU for EX: 0.5264
IoU for SE: 0.5476
IoU for HE: 0.4049
Mean IoU: 0.3847
AP for MA: 0.1267
AP for EX: 0.5524
AP for SE: 0.5888
AP for HE: 0.4649
Mean AP: 0.4332


Epoch 182/200 (Validation):  25%|██▌       | 2/8 [00:19<00:52,  8.68s/it]

In [1]:
import torch
import gc

def clear_gpu_memory():
    # Empty the cache
    torch.cuda.empty_cache()

    # Garbage collect to free up memory
    gc.collect()

    # Print memory statistics before and after clearing
    print("Before clearing:")
    print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

    # Optional: Force PyTorch to release all memory
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            with torch.cuda.device(i):
                torch.cuda.empty_cache()
                torch.cuda.ipc_collect()

    print("\nAfter clearing:")
    print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

    # Environment variable suggestion
    print("\nConsider adding this to your code:")
    print("import os")
    print("os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'")

# Call the function
clear_gpu_memory()

Before clearing:
Allocated: 0.00 GB
Cached: 0.00 GB

After clearing:
Allocated: 0.00 GB
Cached: 0.00 GB

Consider adding this to your code:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def test_model(model_path, test_img_dir, test_mask_dirs, device='cuda'):
    """
    Evaluate trained model on test dataset
    model_path: path to best model weights
    test_img_dir: directory containing test images
    test_mask_dirs: dictionary with paths to test masks for each lesion type
    """
    # Create test dataset
    print("Setting up test dataset from", test_img_dir)
    test_dataset = AdvancedLesionSegmentationDataset(
        image_dir=test_img_dir,
        mask_dirs=test_mask_dirs,
        transform=get_advanced_transforms('val'),  # Use validation transforms
        apply_preprocessing=True
    )

    # Create test dataloader
    test_loader = DataLoader(
        test_dataset,
        batch_size=16,  # Can use larger batch size for testing
        shuffle=False,
        num_workers=8,
        pin_memory=True
    )

    print(f"Test dataset size: {len(test_dataset)}")

    # Load best model (use MAP model as it's better for practical applications)
    best_model = EnhancedUNet(
        encoder_name="resnext101_32x8d",  # Make sure to use same architecture as training
        encoder_weights=None,  # Don't need pretrained weights when loading saved model
        in_channels=3,
        classes=5
    ).to(device)

    best_model.load_state_dict(torch.load(model_path))
    best_model.eval()

    # Setup TTA transforms
    tta_transforms = [
        A.Compose([
            A.Resize(height=512, width=512),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        A.Compose([
            A.Resize(height=512, width=512),
            A.HorizontalFlip(p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        A.Compose([
            A.Resize(height=512, width=512),
            A.VerticalFlip(p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        A.Compose([
            A.Resize(height=512, width=512),
            A.Transpose(p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    ]

    # Define reverse transforms for flips
    def reverse_transform(output, transform_idx):
        if transform_idx == 0:  # Original
            return output
        elif transform_idx == 1:  # HorizontalFlip
            return torch.flip(output, dims=[3])
        elif transform_idx == 2:  # VerticalFlip
            return torch.flip(output, dims=[2])
        elif transform_idx == 3:  # Transpose
            return output.transpose(2, 3)

    # Initialize metrics
    all_ious = []
    all_maps = []
    class_names = ['MA', 'EX', 'SE', 'HE']

    # Create output directory for test results
    os.makedirs('test_results', exist_ok=True)
    os.makedirs('test_predictions', exist_ok=True)

    # Define colormap for visualization
    def create_color_map():
        color_map = np.zeros((5, 3), dtype=np.uint8)
        color_map[0] = [0, 0, 0]      # Background (black)
        color_map[1] = [255, 0, 0]    # MA (red)
        color_map[2] = [0, 255, 0]    # EX (green)
        color_map[3] = [255, 255, 0]  # SE (yellow)
        color_map[4] = [0, 0, 255]    # HE (blue)
        return color_map

    color_map = create_color_map()

    # Function to create colored segmentation
    def create_colored_segmentation(mask):
        colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
        for i in range(5):  # 5 classes including background
            colored_mask[mask == i] = color_map[i]
        return colored_mask

    # Start evaluating
    with torch.no_grad():
        for batch_idx, (images_orig, masks) in enumerate(tqdm(test_loader, desc="Testing")):
            # Clear memory
            torch.cuda.empty_cache()

            # Send masks to device
            masks = masks.to(device).long()

            # Apply TTA and average predictions
            outputs_avg = None

            for t_idx, transform in enumerate(tta_transforms):
                # Apply transform to original image
                if t_idx == 0:
                    # Use the original transformed image
                    images = images_orig.to(device)
                else:
                    # Apply the TTA transform to the original images
                    images_list = []
                    for img in images_orig.cpu().numpy():
                        # Convert from tensor format to numpy for albumentations
                        img = np.transpose(img, (1, 2, 0))  # CHW to HWC
                        # Denormalize
                        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                        img = np.clip(img, 0, 1) * 255
                        img = img.astype(np.uint8)
                        # Apply transform
                        augmented = transform(image=img)
                        images_list.append(augmented['image'])

                    # Convert back to tensor
                    images = torch.stack(images_list).to(device)

                # Forward pass
                outputs = best_model(images)

                # Reverse the transform on outputs
                outputs = reverse_transform(outputs, t_idx)

                # Accumulate outputs
                if outputs_avg is None:
                    outputs_avg = outputs
                else:
                    outputs_avg += outputs

            # Average the outputs
            outputs_avg /= len(tta_transforms)

            # Get predictions
            preds = torch.argmax(outputs_avg, dim=1).cpu().numpy()
            true_masks = masks.cpu().numpy()

            # Calculate IoU with post-processing
            ious = compute_iou(outputs_avg, masks, apply_post_processing=True)
            all_ious.append(ious)

            # Calculate mAP with post-processing
            maps, _ = compute_map(outputs_avg, masks, apply_post_processing=True)
            all_maps.append(maps)

            # Save predictions for each image in the batch
            for i, (pred, true_mask) in enumerate(zip(preds, true_masks)):
                # Create index for the current image
                img_idx = batch_idx * test_loader.batch_size + i

                # Create colored visualizations
                pred_colored = create_colored_segmentation(pred)
                true_colored = create_colored_segmentation(true_mask)

                # Get original image for visualization
                img = images_orig[i].cpu().numpy()
                img = np.transpose(img, (1, 2, 0))  # CHW to HWC
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                img = np.clip(img, 0, 1) * 255
                img = img.astype(np.uint8)

                # Create visualization with original, ground truth, and prediction
                vis_img = np.hstack([img, true_colored, pred_colored])

                # Save visualization
                cv2.imwrite(f'test_predictions/test_img_{img_idx}.png', cv2.cvtColor(vis_img, cv2.COLOR_RGB2BGR))

            # Clean up GPU memory
            del images_orig, masks, outputs_avg
            torch.cuda.empty_cache()

    # Calculate final metrics
    test_ious = np.mean(all_ious, axis=0)
    test_miou = np.mean(test_ious)

    test_maps = np.mean(all_maps, axis=0)
    test_mmap = np.mean(test_maps)

    # Print results
    print("\nTest Set Evaluation Results:")
    for i, lesion_type in enumerate(class_names):
        print(f"IoU for {lesion_type}: {test_ious[i]:.4f}")
    print(f"Mean IoU: {test_miou:.4f}")

    for i, lesion_type in enumerate(class_names):
        print(f"AP for {lesion_type}: {test_maps[i]:.4f}")
    print(f"Mean AP: {test_mmap:.4f}")

    # Save results to file
    with open('test_results/metrics.txt', 'w') as f:
        f.write("Test Set Evaluation Results:\n")
        for i, lesion_type in enumerate(class_names):
            f.write(f"IoU for {lesion_type}: {test_ious[i]:.4f}\n")
        f.write(f"Mean IoU: {test_miou:.4f}\n\n")

        for i, lesion_type in enumerate(class_names):
            f.write(f"AP for {lesion_type}: {test_maps[i]:.4f}\n")
        f.write(f"Mean AP: {test_mmap:.4f}\n")

    # Create confusion matrix for each class
    def generate_confusion_matrices(all_preds, all_true_masks):
        from sklearn.metrics import confusion_matrix
        import seaborn as sns

        # Create 4 confusion matrices (one for each lesion type)
        for c in range(1, 5):  # Skip background class
            all_pred_binary = []
            all_true_binary = []

            # Convert to binary predictions for this class
            for pred, true_mask in zip(all_preds, all_true_masks):
                pred_binary = (pred == c).flatten()
                true_binary = (true_mask == c).flatten()

                all_pred_binary.extend(pred_binary)
                all_true_binary.extend(true_binary)

            # Create confusion matrix
            cm = confusion_matrix(all_true_binary, all_pred_binary)

            # Normalize to get percentages
            cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            cm_norm = np.nan_to_num(cm_norm)

            # Plot confusion matrix
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues')
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.title(f'Confusion Matrix for {class_names[c-1]}')
            plt.savefig(f'test_results/confusion_matrix_{class_names[c-1]}.png')
            plt.close()

    # Collect all predictions and ground truth for confusion matrices
    all_preds = []
    all_true_masks = []

    with torch.no_grad():
        for images, masks in tqdm(test_loader, desc="Collecting predictions"):
            images = images.to(device)
            masks = masks.cpu().numpy()

            # Forward pass with TTA
            outputs_avg = None
            for t_idx in range(1):  # Just use original images for speed
                outputs = best_model(images)
                if outputs_avg is None:
                    outputs_avg = outputs
                else:
                    outputs_avg += outputs

            preds = torch.argmax(outputs_avg, dim=1).cpu().numpy()

            # Add to lists
            all_preds.extend(preds)
            all_true_masks.extend(masks)

            # Clean memory
            del images, masks, outputs, outputs_avg
            torch.cuda.empty_cache()

    # Generate confusion matrices
    try:
        generate_confusion_matrices(all_preds, all_true_masks)
        print("Confusion matrices saved to test_results directory")
    except Exception as e:
        print(f"Could not generate confusion matrices: {e}")

    return test_miou, test_mmap

if __name__ == "__main__":
    # Paths for test dataset
    test_img_dir = "/content/DDR_dataset/DDR-dataset/lesion_segmentation/test/image"
    test_mask_dirs = {
        "MA": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/test/label/MA",
        "EX": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/test/label/EX",
        "SE": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/test/label/SE",
        "HE": "/content/DDR_dataset/DDR-dataset/lesion_segmentation/test/label/HE"
    }

    # Path to best model weights (use the MAP one as it's typically better for practical applications)
    model_path = 'models/best_model_map.pth'

    # Run testing on best model
    test_miou, test_mmap = test_model(model_path, test_img_dir, test_mask_dirs, device=device)

    print(f"\nFinal Test Results:")
    print(f"Mean IoU on Test Set: {test_miou:.4f}")
    print(f"Mean AP on Test Set: {test_mmap:.4f}")

Setting up test dataset from /content/DDR_dataset/DDR-dataset/lesion_segmentation/test/image
Test dataset size: 225


Testing: 100%|██████████| 15/15 [01:13<00:00,  4.92s/it]



Test Set Evaluation Results:
IoU for MA: 0.0382
IoU for EX: 0.2609
IoU for SE: 0.6607
IoU for HE: 0.2898
Mean IoU: 0.3124
AP for MA: 0.1094
AP for EX: 0.3154
AP for SE: 0.6828
AP for HE: 0.3612
Mean AP: 0.3672


Confusion matrices saved to test_results directory

Final Test Results:
Mean IoU on Test Set: 0.3124
Mean AP on Test Set: 0.3672
